На вход поступает файл, в котором позиции в чеках заведомо распределены некоторым образом по кластерам. Проанализируйте текущее распределение, попытайтесь выбрать метрику(и) близости кластеров и произвести перераспределение объектов. Число кластеров может быть изменено.

# Загрузка ембединга

In [3]:
import gensim.downloader as api

ru_word2vec_model = api.load("word2vec-ruscorpora-300")

[==================================================] 100.0% 198.8/198.8MB downloaded


In [4]:
import pickle

with open("ru_word2vec_300.pickle", "wb") as file:
    pickle.dump(ru_word2vec_model, file)

In [26]:
embeddiing_path = "ru_word2vec_300.pickle"

def open_embedding(path: str):
    with open(path, "rb") as file:
        return pickle.load(file)
    
embedding_300_dim = open_embedding(embeddiing_path)

# Зависимости

In [143]:
import re
import annoy
import numpy as np
import pandas as pd
from loguru import logger

import torch
from torch.utils.data import Dataset

import pymorphy2
from nltk.stem.snowball import RussianStemmer
from nltk import word_tokenize
from nltk.corpus import stopwords

from typing import List, Tuple
from itertools import zip_longest
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import silhouette_score

In [2]:
file_path = 'test_task_NLP.json'
df = pd.read_json(file_path)

In [3]:
df

,0,1,2,3,4,5,6
0,Баклажаны с орехами упак,Баклажаны с творожной начинкой от бренд-шефа АВ,None,None,None,None,None
1,Блинчики с курицей упак (4 шт),"Блинчики с мясом, Уже Готово , 140 г",None,None,None,None,None
2,Блинчики с мясом упак (4 шт),"Блинчики с мясом, Уже Готово , 220 г, Россия",None,None,None,None,None
3,Блины без начинки упак,"Блины Русские от бренд-шефа АВ, Россия",None,None,None,None,None
4,Винегрет упак,Салат FreshSecret Винегрет 600г,"Винегрет с ароматным подсолнечным маслом, Уже ...",None,None,None,None
...,...,...,...,...,...,...,...
1218,Эскимо Чистая Линия Шоколадное пломбир в молоч...,"Мороженое Советское шоколадное, Чистая линия ,...",None,None,None,None,None
1219,Яйцо куриное Праксики с селеном 6 шт Россия,Яйцо куриное Праксики С1 6шт ОАО Солигорская п...,None,None,None,None,None
1220,Яйцо куриное Праксис отборное 10шт Россия,Яйцо куриное Праксис столовое С0 коричневое 10шт,None,None,None,None,None
1221,Яйцо куриное Праксис С1 10шт Россия,Яйцо куриное Праксис столовое С1 коричневое 10шт,None,None,None,None,None


# Анализ текущего распределения

## Проверка на дубликаты

### Дубликатов среди всех записей нет

In [4]:
df.duplicated().sum()

0

### "Шоколад Lindt Lindor, молочный, 200г" сразу объеденяет 8 продуктов (Дубликат в 0 колонке)

In [5]:
duplicates = df.duplicated(subset=[0])
duplicates[duplicates].index

Int64Index([625], dtype='int64')

In [6]:
copies = df[df.iloc[:, 0] == df.iloc[625,0]]
copies

,0,1,2,3,4,5,6
623,"Шоколад Lindt Lindor, молочный, 200г",Конфеты Lindt Линдор Ассорти 200г Италия,Конфеты Lindor шоколадные Ассорти с начинкой 200г,Шоколадный набор Lindt Lindor Ассорти горький/...,"Шоколадные конфеты Lindor Ассорти , Lindt, 200...",None,None
625,"Шоколад Lindt Lindor, молочный, 200г",Конфеты Lindt Линдор Молочный шоколад 200 г Ит...,"Шоколадные конфеты LINDT LINDOR Ассорти, 200г",Шоколадный набор Lindt Lindor молочный с начин...,Конфеты Lindor из молочного шоколада с начинко...,Шоколадные конфеты Lindt Lindor молочный шокол...,None


## Анализ некоторых результатов распределения

### Распределение не видит схожести между блинами и блинчиками, но видит не сущестующие разлиция между блинчиками

In [7]:
df.iloc[1:4, :]

,0,1,2,3,4,5,6
1,Блинчики с курицей упак (4 шт),"Блинчики с мясом, Уже Готово , 140 г",None,None,None,None,None
2,Блинчики с мясом упак (4 шт),"Блинчики с мясом, Уже Готово , 220 г, Россия",None,None,None,None,None
3,Блины без начинки упак,"Блины Русские от бренд-шефа АВ, Россия",None,None,None,None,None


### Так как алгоритм разделил яйца на 4 кластера, он обращает внимание на несущественную информацию. 

In [8]:
df.iloc[1219:, :]

,0,1,2,3,4,5,6
1219,Яйцо куриное Праксики с селеном 6 шт Россия,Яйцо куриное Праксики С1 6шт ОАО Солигорская п...,None,None,None,None,None
1220,Яйцо куриное Праксис отборное 10шт Россия,Яйцо куриное Праксис столовое С0 коричневое 10шт,None,None,None,None,None
1221,Яйцо куриное Праксис С1 10шт Россия,Яйцо куриное Праксис столовое С1 коричневое 10шт,None,None,None,None,None
1222,Яйцо Праксис перепелиное 20 шт Россия,"Яйца перепелиные Праксис, 20 штук",None,None,None,None,None


### Примеры правильной кластеризации

In [9]:
df.iloc[4:10, :]

,0,1,2,3,4,5,6
4,Винегрет упак,Салат FreshSecret Винегрет 600г,"Винегрет с ароматным подсолнечным маслом, Уже ...",None,None,None,None
5,Голубцы мясные тушеные в томатно-сливочном соу...,Голубцы мясные упак,Голубцы мясные упак,None,None,None,None
6,Запеканка творожная President Дольче ванильная...,Запеканка творожная упак,"Запеканка творожная с изюмом, Уже Готово , 300 г",None,None,None,None
7,Морковь по-корейски,Морковь по-корейски упак,"Салат FreshSecret Морковь по-корейски, 250г","Морковь по-корейски, Уже Готово , 250 г",None,None,None
8,Оладьи из кабачков упак,"Оладьи из кабачков, Уже Готово , 250 г",None,None,None,None,None
9,Плов с мясом баранины упак,Плов узбекский от бренд-шефа АВ,None,None,None,None,None


In [10]:
type(df.iloc[4:5, 6].item())

NoneType

# Выводы

- из-за недостатка кластеров есть 1 дубликат (или использование аппроксимированных методов)
- алгоритм не видит схожесть в близких словах (решение: стемминг, лемматизация)
- несущественная информация путает алгорим (решение: удаление через стоп слова и регулярные выражения)
- обращает внимание на колличество штук товара, марку производителя, вес

# Подготовка данных

### Создадим класс для удобного анализа и тренеровки алгоритмов

In [12]:
class TextData(Dataset):
    
    def __preprocess_data(self, preclustered_data: pd.DataFrame) -> List[str]:
        pure_data = []
        
        for index, row in preclustered_data.iterrows():
            for value in row:
                if value is not None:
                    pure_data.append(value.lower())
                    
        return list(set(pure_data)) # set conversion for duplicate deletion
    
    def __init__(self, preclustered_data: pd.DataFrame) -> None:
        self.preclustered_data = preclustered_data
        self.data = self.__preprocess_data(self.preclustered_data)
        
    def __len__(self) -> int:
        return len(self.data)
    
    def __getitem__(self, idx) -> str:
        return self.data[idx]

### Расширим функциональность и добавим обработку текста и его векторицацию `__getitem__` удобен для просмотра всех результатов

In [18]:
class CleanTextData(TextData):
        
    def __apply_reg_ex(self, data: List[str], reg_ex_patterns: List[str]) -> List[str]:
        clear_data = []
        replacement = ""
        
        for product in data:
            processed = product
            for pattern in reg_ex_patterns:
                processed = re.sub(pattern, replacement, processed)
            clear_data.append(processed)
            
        return clear_data
    
    def __tokenize(self, data: List[str], delete_stop_words = True) -> List[List[str]]:
        language = 'russian'
        tokenized_data = []
        stop_words = set(stopwords.words(language))
        
        for product in data:
            words = word_tokenize(product, language=language)
            if delete_stop_words:
                filtered_words = [word for word in words if word.casefold() not in stop_words]
                tokenized_data.append(filtered_words)
            else:
                tokenized_data.append(words)
            
        return tokenized_data
    
    def __lemmatize(self, data: List[List[str]]) -> List[List[str]]:
        language = 'ru'
        morph = pymorphy2.MorphAnalyzer(lang=language)
        lemmatized_data = []
        
        for product in data:
            lemmas = [morph.parse(word)[0].normal_form for word in product]
            lemmatized_data.append(lemmas)
            
        return lemmatized_data
    
    def __apply_preprocessing(self) -> List[List[str]]:
        reg_ex_data = self.__apply_reg_ex(self.data, self.reg_ex_patterns)
        tokenized_data = self.__tokenize(reg_ex_data)
        lemmatized_data = self.__lemmatize(tokenized_data)
        return lemmatized_data
    
    def __generate_tf_idf(self) -> List[List[int]]:
        tfidf_vectorizer = TfidfVectorizer()
        string_list = [' '.join(word_list) for word_list in self.tokenized_data]
        return tfidf_vectorizer.fit_transform(string_list).toarray()   
    
    def __convert_vectors(self):
        return self.__generate_tf_idf()
    
    def __init__(self, preclustered_data: pd.DataFrame, reg_ex_patterns: List[str], vectotization_method: str) -> None:
        super().__init__(preclustered_data)
        self.reg_ex_patterns = reg_ex_patterns
        self.tokenized_data = self.__apply_preprocessing()
        self.vectorized_data = self.__convert_vectors()
        
    def __getitem__(self, idx) -> Tuple[List[str], List[List[str]], np.array]:
        return self.data[idx], self.tokenized_data[idx], self.vectorized_data[idx]

### Определим регулярные выражения для очистки данных

In [19]:
reg_ex_patterns = {
    "numbers": r"\d+",
    "stand_alone_g": "(?<![a-zA-Zа-яА-Я])г(?![a-zA-Zа-яА-Я])",
    "comma": r",",
    "dot": r".",
    "english_letters": r"[a-zA-Z]+",
}


final_reg_ex_pattern = "[a-zA-Z]+|\d+|(?<![a-zA-Zа-яА-Я])г(?![a-zA-Zа-яА-Я])|,|``|''|мл"

### Посмотрим результаты

In [20]:
cleanTextData = CleanTextData(df, [final_reg_ex_pattern], "tf-idf")

In [21]:
len(cleanTextData)

3440

In [22]:
cleanTextData[0]

('шоколад ritter sport с кокосовой начинкой 100г',
 ['шоколад', 'кокосовый', 'начинка'],
 array([0., 0., 0., ..., 0., 0., 0.]))

# Моделирование

In [101]:
class ClusteringNearestNeighborsModel():
    
    def __init__(self, data: CleanTextData, n_neighbors: int, algorithm: str, metric: str) -> None:
        self.data = data
        self.model = NearestNeighbors(n_neighbors=n_neighbors, algorithm=algorithm, metric=metric)
        
    def clusterize(self) -> np.array:
        self.model.fit(self.data.vectorized_data)
        distances, indices = self.model.kneighbors(self.data.vectorized_data)
        return np.unique(indices, axis=0)
   
    def cluster_to_text(self, clusters: np.array) -> List[List[str]]:
        text_clusters = []
        
        for i in range(clusters.shape[0]):
            individual_cluster = []
            for j in range(clusters.shape[1]):
                individual_cluster.append(self.data[clusters[i][j]][0])
            text_clusters.append(individual_cluster)
        
        return text_clusters
    
    @staticmethod
    def list_to_dataframe(text_clusters: List[List[str]]) -> pd.DataFrame:
        return pd.DataFrame(text_clusters)

### Выберем колличество соседей, алгоритм и метрику

In [102]:
n_neighbors = 6
algorithm = 'auto'
metric = 'cosine'

knn = ClusteringNearestNeighborsModel(cleanTextData, n_neighbors, algorithm, metric)

In [97]:
clustered_data = knn.clusterize()
clustered_data

array([[   0, 2634,  345, 2597,  363,   97],
       [   1, 1973,  163,  664, 2950, 1717],
       [   2, 1289, 2986, 2045, 2234, 1907],
       ...,
       [3436,  956, 3146, 2994, 1169, 1847],
       [3437,  824, 2248,   66, 2698, 1878],
       [3438, 1697, 2227, 1651, 1571, 2496]], dtype=int64)

In [98]:
text_clusters = knn.cluster_to_text(clustered_data)
text_clusters

[['шоколад ritter sport с кокосовой начинкой 100г',
  'конфеты lindor из молочного шоколада с начинкой 200г',
  'шоколад ritter sport, молочный с начинкой и печеньем, 100г',
  'шоколад ritter sport молочный с начинкой клубника с йогуртом 100г',
  'шоколад ritter sport молочный альпийское молоко и начинка какао 100г германия',
  'пирожное kinder pingui бисквитное, покрытое шоколадом, с молочной начинкой 30г'],
 ['изюм семушка узбекский черный 150г',
  'изюм семушка узбекский 150г россия',
  'смесь семушка жареных орехов и изюма, 250г',
  'смесь семушка жареных орехов и изюма 250г россия',
  'смесь жареных орехов и изюма, семушка , 250 г, россия',
  'чай черный greenfield golden ceylon, 25х2г'],
 ['журнал "караван историй. коллекция"',
  'журнал караван историй россия',
  'журнал star hit',
  'журнал "grazia"',
  "журнал men's health",
  'журнал glamour'],
 ['пиво бавария премиум пилзнер светлое ст/б 0.5л россия',
  'пиво светлое pilsner urquell пилзнер стекло, 0,5л',
  'пиво velkopopovi

In [99]:
pd_text_clusters = ClusteringNearestNeighborsModel.list_to_dataframe(text_clusters)
pd_text_clusters

,0,1,2,3,4,5
0,шоколад ritter sport с кокосовой начинкой 100г,конфеты lindor из молочного шоколада с начинко...,"шоколад ritter sport, молочный с начинкой и пе...",шоколад ritter sport молочный с начинкой клубн...,шоколад ritter sport молочный альпийское молок...,"пирожное kinder pingui бисквитное, покрытое шо..."
1,изюм семушка узбекский черный 150г,изюм семушка узбекский 150г россия,"смесь семушка жареных орехов и изюма, 250г",смесь семушка жареных орехов и изюма 250г россия,"смесь жареных орехов и изюма, семушка , 250 г,...","чай черный greenfield golden ceylon, 25х2г"
2,"журнал ""караван историй. коллекция""",журнал караван историй россия,журнал star hit,"журнал ""grazia""",журнал men's health,журнал glamour
3,пиво бавария премиум пилзнер светлое ст/б 0.5л...,"пиво светлое pilsner urquell пилзнер стекло, 0,5л",пиво velkopopovicky kozel премиум светлое ст/б...,"пиво leffe blonde светлое 6,6% 0,33л ст/б","пиво weihenstephaner светлое ст/б 0,5л германия","пиво warsteiner светлое ст/б 0,5л германия"
4,колбаса alto concetto фуэт пикантный с/в в/с 1...,колбаса alto concetto фуэт с/в в/с 170г россия,колбаса сыровяленная alto фуэт пикантный тд ви...,колбаса с/в casademont фуэт экстра 150г россия,колбаса фуэт экстра с/в casademont 150 г россия,колбаса с/в casademont фуэт экстра с инжиром 1...
...,...,...,...,...,...,...
2723,"лук репчатый молодой, 2,3-2,5кг. урожай 2015 года",лук репчатый кг,лук репчатый,лук репчатый россия,"лук репчатый (сетка), 5кг",капуста молодая россия
2724,йогурт питьевой агуша детский персик 2.7% 200г...,"йогурт питьевой агуша персик 2,7%, 200г",йогурт питьевой агуша персик 2.7% 200г,"йогурт питьевой агуша персик 2,7% 200г","йогурт питьевой агуша детский натуральный 3,1%...","творог детский агуша персик 3,9% 100г россия"
2725,творог слоеный агуша я сам клубника - ваниль д...,"творог детский я сам. клубника-ваниль 3.8%, аг...",творог фруктовый агуша я сам клубника/ваниль 3...,творог слоеный агуша я сам малина-банан-печень...,творог агуша я сам фруктовый 2-сл клубника/ван...,"творог агуша классический 4,5% с 6 месяцев 100г"
2726,сок gerber яблочно-грушевый без сахара с 4 мес...,"сок gerber грушевый без сахара с 4 месяцев, 175г",сок gerber грушевый с 4 месяцев 175г польша,йогурт питьевой агуша яблочно-грушевый 2.7% 20...,сок fleur alpine грушевый с 4 месяцев 200г сте...,"сок грушевый осветленный с 4 месяцев, фрутонян..."


## Approximate nearest neighbors метод если нужна скорость работы

In [86]:
class ClusteringApproximateNearestNeighborsModel():
    
    def __init__(self, data: CleanTextData, n_neighbors: int, n_trees: int, search_k: int, metric: str) -> None:
        self.data = data
        self.n_neighbors = n_neighbors
        self.search_k = search_k
        self.ann_index = annoy.AnnoyIndex(self.data.vectorized_data.shape[1], metric=metric)
        
    def clusterize(self) -> np.array:
        clusters = []
        
        for i in range(self.data.vectorized_data.shape[0]):
            self.ann_index.add_item(i, X[i])

        try:
            self.ann_index.build(n_trees)
        except ValueError:
            logger.info("index has alreary been built")
        
        for i in range(len(self.data)):
            neighbors, distances = ann_index.get_nns_by_item(i, self.n_neighbors, search_k=search_k, include_distances=True)
            clusters.append(neighbors)
        
        return np.unique(np.array(clusters), axis=0)
   
    def cluster_to_text(self, clusters: np.array) -> List[List[str]]:
        text_clusters = []
        
        for i in range(clusters.shape[0]):
            individual_cluster = []
            for j in range(clusters.shape[1]):
                individual_cluster.append(self.data[clusters[i][j]][0])
            text_clusters.append(individual_cluster)
        
        return text_clusters
    
    @staticmethod
    def list_to_dataframe(text_clusters: List[List[str]]) -> pd.DataFrame:
        return pd.DataFrame(text_clusters)

In [87]:
ann = ClusteringApproximateNearestNeighborsModel(cleanTextData, n_neighbors=8, n_trees=100, search_k=10, metric="angular")

In [88]:
clustered_data = ann.clusterize()
clustered_data

array([[   0, 2634,  345, ...,   97,   16, 1334],
       [   1,  163,  664, ..., 1717, 2089, 1149],
       [   2, 1289,  552, ..., 2234, 2391, 2959],
       ...,
       [3436,  956, 3146, ..., 1847, 2362, 2882],
       [3437,  824, 2248, ..., 2453, 2477, 2873],
       [3438, 1697, 2227, ..., 2496, 3311, 2458]])

In [91]:
text_clustered_data = ann.cluster_to_text(clustered_data)
text_clustered_data

[['шоколад ritter sport с кокосовой начинкой 100г',
  'конфеты lindor из молочного шоколада с начинкой 200г',
  'шоколад ritter sport, молочный с начинкой и печеньем, 100г',
  'шоколад ritter sport молочный с начинкой клубника с йогуртом 100г',
  'шоколад ritter sport молочный альпийское молоко и начинка какао 100г германия',
  'пирожное kinder pingui бисквитное, покрытое шоколадом, с молочной начинкой 30г',
  'конфеты raffaello с цельным миндальным орехом в кокосовой обсыпке, 240г',
  'конфеты raffaello с цельным миндальным орехом в кокосовой обсыпке 150г'],
 ['изюм семушка узбекский черный 150г',
  'смесь семушка жареных орехов и изюма, 250г',
  'смесь семушка жареных орехов и изюма 250г россия',
  'смесь жареных орехов и изюма, семушка , 250 г, россия',
  'чай черный greenfield earl grey fantasy 100г',
  'чай черный greenfield golden ceylon, 25х2г',
  'чай richard royal ceylon черный 100х2г',
  'плов узбекский от бренд-шефа ав'],
 ['журнал "караван историй. коллекция"',
  'журнал ка

In [92]:
ClusteringApproximateNearestNeighborsModel.list_to_dataframe(text_clustered_data)

,0,1,2,3,4,5,6,7
0,шоколад ritter sport с кокосовой начинкой 100г,конфеты lindor из молочного шоколада с начинко...,"шоколад ritter sport, молочный с начинкой и пе...",шоколад ritter sport молочный с начинкой клубн...,шоколад ritter sport молочный альпийское молок...,"пирожное kinder pingui бисквитное, покрытое шо...",конфеты raffaello с цельным миндальным орехом ...,конфеты raffaello с цельным миндальным орехом ...
1,изюм семушка узбекский черный 150г,"смесь семушка жареных орехов и изюма, 250г",смесь семушка жареных орехов и изюма 250г россия,"смесь жареных орехов и изюма, семушка , 250 г,...",чай черный greenfield earl grey fantasy 100г,"чай черный greenfield golden ceylon, 25х2г",чай richard royal ceylon черный 100х2г,плов узбекский от бренд-шефа ав
2,"журнал ""караван историй. коллекция""",журнал караван историй россия,журнал gq,журнал glamour,"журнал ""grazia""",журнал men's health,"журнал ""robb report""",журнал forbes
3,пиво бавария премиум пилзнер светлое ст/б 0.5л...,"пиво светлое pilsner urquell пилзнер стекло, 0,5л",пиво velkopopovicky kozel премиум светлое ст/б...,"пиво leffe blonde светлое 6,6% 0,33л ст/б","пиво clausthaler classic светлое б/а ст/б 0,33...","пиво weihenstephaner светлое ст/б 0,5л германия","пиво kulmbacher светлое ст/б 0,5л германия",пиво paulaner original munchner l светлое ст/б...
4,колбаса alto concetto фуэт пикантный с/в в/с 1...,колбаса alto concetto фуэт с/в в/с 170г россия,колбаса сыровяленная alto фуэт пикантный тд ви...,колбаса фуэт экстра с/в casademont 150 г россия,колбаса с/в casademont фуэт экстра 150г россия,колбаса фуэт экстра с/в casademont с инжиром 1...,колбаса с/в casademont фуэт экстра с инжиром 1...,колбаса сыровяленная alto фуэт тд вик 170г россия
...,...,...,...,...,...,...,...,...
2703,"лук репчатый молодой, 2,3-2,5кг. урожай 2015 года",лук репчатый кг,лук репчатый,лук репчатый россия,"лук репчатый (сетка), 5кг",лук-порей,картофель молодой азербайджан,чипсы lays сметана и лук 150г
2704,йогурт питьевой агуша детский персик 2.7% 200г...,"йогурт питьевой агуша персик 2,7% 200г",йогурт питьевой агуша персик 2.7% 200г,"йогурт питьевой агуша персик 2,7%, 200г","йогурт питьевой агуша детский натуральный 3,1%...","творог детский агуша персик 3,9% 100г россия","йогурт агуша персик 2,7% с 8 месяцев 200г",творог детский агуша персик 3.9% 100г
2705,творог слоеный агуша я сам клубника - ваниль д...,"творог детский я сам. клубника-ваниль 3.8%, аг...",творог фруктовый агуша я сам клубника/ваниль 3...,творог слоеный агуша я сам малина-банан-печень...,творог агуша я сам фруктовый 2-сл клубника/ван...,"творог агуша классический 4,5% с 6 месяцев 100г","творог слоеный агуша малина/банан/печенье 3,8%...","творог агуша персик с 6 месяцев 3,9% 100г"
2706,сок gerber яблочно-грушевый без сахара с 4 мес...,"сок gerber грушевый без сахара с 4 месяцев, 175г",сок gerber грушевый с 4 месяцев 175г польша,сок fleur alpine грушевый с 4 месяцев 200г сте...,"сок грушевый осветленный с 4 месяцев, фрутонян...",сок gerber яблочно-виноградный с шиповником с ...,сок фрутоняня яблочно-персиковый неосветленный...,сок gerber яблочно-морковный с мякотью с 5-ти ...


## k means подход

In [161]:
class KmeansTuned():
    
    def __append_none_to_lists(self, groups: dict) -> dict:
        max_len = max(len(v) for v in groups.values())
        
        for k in groups.keys():
            if len(groups[k]) < max_len:
                groups[k].extend([None] * (max_len - len(groups[k])))
    
    def tune_n_clusters(self) -> np.array:
        best_score = -1
        best_n = 0
        best_labels = None
        
        for i in self.n_clusters:
            kmeans = KMeans(n_clusters=i)
            kmeans.fit(cleanTextData.vectorized_data)
            labels = kmeans.labels_

            score = silhouette_score(self.data.vectorized_data, labels, metric=self.metric, random_state=123)
            logger.info(f"number of clusters: {i}, silhouette score: {score}")
            if score > best_score:
                best_score = score
                best_n = i
                best_labels = labels
        
        return best_labels
    
    def __init__(self, data: CleanTextData, n_clusters: List[int], metric: str) -> None:
        self.data = data
        self.n_clusters = n_clusters
        self.metric = metric
        
    def get_clusterezation(self, best_labels) -> pd.DataFrame:
        groups = {}
        initial_text = self.data.data
        tuples = list(zip(initial_text, best_labels))
        
        for tup in tuples:
            val = tup[1]
            if val not in groups:
                groups[val] = []
            groups[val].append(tup[0])
            
        self.__append_none_to_lists(groups)
            
        return pd.DataFrame(groups).T.sort_index()

In [165]:
n_clusters = [200, 300, 600]
metric = 'cosine'

kmeans = KmeansTuned(cleanTextData, n_clusters, metric)

In [166]:
best_labels = kmeans.tune_n_clusters()
best_labels

2023-04-14 15:56:44.320 | INFO     | __main__:tune_n_clusters:21 - number of clusters: 200, silhouette score: 0.2752544576274336
2023-04-14 15:57:46.034 | INFO     | __main__:tune_n_clusters:21 - number of clusters: 300, silhouette score: 0.3171516492929135
2023-04-14 15:59:48.597 | INFO     | __main__:tune_n_clusters:21 - number of clusters: 600, silhouette score: 0.4147861086067348


array([406,  26, 574, ..., 481, 198,  47])

In [167]:
kmeans.get_clusterezation(best_labels)

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,филе утолина грудки утёнка без кожи замороженн...,"окорочок утолина утёнка с кожей замороженный, ...",филе грудки утенка охл. утолина россия,филе утолина грудки утёнка с кожей замороженно...,филе грудки утенка б/кожи охл. утолина россия,филе грудки утенка утолина замороженное,филе грудки утиное охлажденное б/кожи утолина кг,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,чай richard royal ceylon черн 100*2г россия,чай ahmad tea летний чабрец черн 100г оаэ,чай ahmad tea летний чабрец черн 25*1.5г россия,чай ahmad tea черн английский завтрак к/к 100г...,чай greenfield голден цейлон черн 100г россия,"чай greenfield спринг мелоди черн 25*1,5г россия",чай greenfield голден цейлон черн 25*2г россия,чай newby английский завтрак черн 25*2г англия,None,None,...,None,None,None,None,None,None,None,None,None,None
2,вода s.pellegrino минеральная природная лечебн...,вода боржоми минеральная лечебно-столовая гази...,вода s.pellegrino минеральная природная лечебн...,вода borjomi минеральная природная питьевая ле...,вода borjomi минеральная природная питьевая ле...,вода минеральная боржоми лечебно-столовая 0.33...,вода borjomi минеральная природная питьевая ле...,вода borjomi минеральная природная питьевая ле...,вода боржоми минеральная лечебно-столовая гази...,вода боржоми минеральная лечебно-столовая гази...,...,None,None,None,None,None,None,None,None,None,None
3,"йогурт братья чебурашкины клубника 0,5% , 330г","йогурт питьевой братья чебурашкины 0,5% класси...","йогурт братья чебурашкины питьевой клубника 0,...","йогурт братья чебурашкины 0,5% малина, 330 г","йогурт братья чебурашкины малина 0,5% , 330г","йогурт братья чебурашкины питьевой лимон 0,5% ...","йогурт питьевой братья чебурашкины 0,5% клубни...","йогурт питьевой малина 0.5%, братья чебурашкин...",йогурт питьевой братья чебурашкины натуральный...,"йогурт братья чебурашкины питьевой малина 0,5%...",...,None,None,None,None,None,None,None,None,None,None
4,кефир агуша для детского питания с 8 месяцев 3...,мыло johnson's baby детское с молоком с детски...,"молоко детское агуша 3,2% 500г","кефир детский агуша 3,2% 204г россия",кефир детский агуша классический 3.2% 204мл,"кефир агуша дет 3,2% 204г tba",молоко детское агуша витаминизированное 2.5% 2...,"кефир агуша детский 3,2% 204г россия","молоко агуша источник кальция с 8 месяцев 2,5%...",бифидокефир детский агуша 3.2% 204мл,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,"йогурт агуша вязкий земляника/малина 2,7% 90г",йогурт питьевой агуша я сам малина 2.7% 200г,йогурт питьевой агуша персиковый 2.7% 200 г ро...,"йогурт тема питьевой банан/земляника 2,8% 210г...","йогурт тема питьевой шиповник/малина 2,8% 210г...","йогурт питьевой агуша я сам малина 2,7% 200г р...","йогурт агуша малина/земляника 2,7% 90г россия","йогурт питьевой тема с малиной и шиповником 2,...",None,None,...,None,None,None,None,None,None,None,None,None,None
596,хлопья nordic овсяные органические 600г финляндия,каша svalia овсяная 6% 200г,хлопья nordic органик овсяные 600 г финляндия,"биойогурт питьевой danone активиа 1,3% с банан...",None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
597,мороженое 48 копеек шоколад с шоколадным соусо...,мороженое nestle 48 копеек шоколадное с шок со...,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
598,диски чистоты туалетный утенок океанский оазис...,средство по уходу за туалетом туалетный утенок...,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


# Выводы

- Очистка данных и нижний регистр крайне желательны
- Класс `CleanTextData` расширяем и могут быть добавленны например ембеддинги вместо tf-idf
- Если важна скорость работы можно использовать ANN вместо KNN
- Колличество соседей в размере 8 лучше, чем 5. Много похожий товаров.
- Классы можно внедрять в продакт, добавив валидацию данных
- Kmeans класс дает строгую оценку и имеет возможность тюнинга (можно исполозовать его в паре с ANN или KNN)
- В Kmeans значение 600 является компромисным так как имеет нормальный силует скор и максимум 37 элементов